# Project1
## Momentum Trading
### MACD

In [1]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
#from MCForecastTools import MCSimulation
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import hvplot.pandas


%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

### Collect Apple and SP500 Prices Using the `requests` Library

In [3]:
# Set current amount of stock assets
my_apple_1time=1
my_sp500_1time=1

In [4]:
# stock API URLs
my_apple_1time_url = "https://api.alternative.me/v2/ticker/?symbol=AAPL"
my_sp500_1time_url = "https://api.alternative.me/v2/ticker/?symbol=SP500"

In [5]:
# Fetch current Apple price
apple_price=requests.get(my_apple_1time_url).json()
apple_price



{'data': {'1': {'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'website_slug': 'bitcoin',
   'rank': 1,
   'circulating_supply': 19534562,
   'total_supply': 19534562,
   'max_supply': 21000000,
   'quotes': {'USD': {'price': 34891.0,
     'volume_24h': 12137391746,
     'market_cap': 681507362017,
     'percentage_change_1h': -0.349600144402203,
     'percentage_change_24h': -0.789163746753598,
     'percentage_change_7d': 1.78710165773152,
     'percent_change_1h': -0.349600144402203,
     'percent_change_24h': -0.789163746753598,
     'percent_change_7d': 1.78710165773152}},
   'last_updated': 1699246351},
  '1027': {'id': 1027,
   'name': 'Ethereum',
   'symbol': 'ETH',
   'website_slug': 'ethereum',
   'rank': 2,
   'circulating_supply': 120269158,
   'total_supply': 120269158,
   'max_supply': 120269158,
   'quotes': {'USD': {'price': 1877.67,
     'volume_24h': 11423737317,
     'market_cap': 225793131138,
     'percentage_change_1h': -0.647035096046464,
     'percentage_

In [6]:
# Fetch current SP500 price
sp500_price=requests.get(my_sp500_1time_url).json()
sp500_price



{'data': {'1': {'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'website_slug': 'bitcoin',
   'rank': 1,
   'circulating_supply': 19534562,
   'total_supply': 19534562,
   'max_supply': 21000000,
   'quotes': {'USD': {'price': 34891.0,
     'volume_24h': 12137391746,
     'market_cap': 681507362017,
     'percentage_change_1h': -0.349600144402203,
     'percentage_change_24h': -0.789163746753598,
     'percentage_change_7d': 1.78710165773152,
     'percent_change_1h': -0.349600144402203,
     'percent_change_24h': -0.789163746753598,
     'percent_change_7d': 1.78710165773152}},
   'last_updated': 1699246351},
  '1027': {'id': 1027,
   'name': 'Ethereum',
   'symbol': 'ETH',
   'website_slug': 'ethereum',
   'rank': 2,
   'circulating_supply': 120269158,
   'total_supply': 120269158,
   'max_supply': 120269158,
   'quotes': {'USD': {'price': 1877.67,
     'volume_24h': 11423737317,
     'market_cap': 225793131138,
     'percentage_change_1h': -0.647035096046464,
     'percentage_

In [7]:
# Compute current value of my crpto
my_apple_value=my_apple_1time*apple_price['data']['1']['quotes']['USD']['price']
my_sp500_value=my_sp500_1time*sp500_price['data']['1027']['quotes']['USD']['price']

# Print current crypto wallet balance
print(f"The current value of your {my_apple_1time} AAPLE is ${my_apple_value:0.2f}")
print(f"The current value of your {my_sp500_1time} SP500 is ${my_sp500_value:0.2f}")

The current value of your 1 AAPLE is $34891.00
The current value of your 1 SP500 is $1877.67


### Collect Investments Data Using Alpaca: `AAPLE` (stocks) and `SP500` (stocks)

In [8]:
# Set current amount of shares
my_apple = 1
my_sp500 = 1

In [9]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [10]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"

# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_tickers.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.90,169.00,168.60,391,168.98,15303,168.871369,AAPL
2023-10-30 08:15:00+00:00,168.85,169.00,168.85,105,168.99,5249,168.955731,AAPL
2023-10-30 08:30:00+00:00,168.85,168.92,168.85,72,168.85,3911,168.863544,AAPL
2023-10-30 08:45:00+00:00,168.99,168.99,168.71,81,168.86,4184,168.869565,AAPL
2023-10-30 09:00:00+00:00,169.23,169.23,168.97,111,168.97,5241,169.137236,AAPL


In [11]:
# Reorganize the DataFrame
# Separate ticker data
df_tickers.drop(['high','low','trade_count','open','volume','vwap','symbol'], axis=1, inplace=True)

df_tickers.head()

,close
timestamp,
2023-10-30 08:00:00+00:00,168.90
2023-10-30 08:15:00+00:00,168.85
2023-10-30 08:30:00+00:00,168.85
2023-10-30 08:45:00+00:00,168.99
2023-10-30 09:00:00+00:00,169.23


### MACD formula

In [12]:
# Set MACD formula and iterate over Apple data for the specified intervals and period of time

moving_average_convergance_divergence_ema12 = df_tickers['close'].ewm(halflife=12).mean()
moving_average_convergance_divergence_ema26 = df_tickers['close'].ewm(halflife=26).mean()
  

In [13]:

MACD_line = moving_average_convergance_divergence_ema12 - moving_average_convergance_divergence_ema26
MACD_line.head()


timestamp
2023-10-30 08:00:00+00:00    0.000000
2023-10-30 08:15:00+00:00   -0.000389
2023-10-30 08:30:00+00:00   -0.000511
2023-10-30 08:45:00+00:00    0.001111
2023-10-30 09:00:00+00:00    0.005153
Name: close, dtype: float64

In [14]:
Signal_line = MACD_line.ewm(halflife=9).mean()
Signal_line.head()

timestamp
2023-10-30 08:00:00+00:00    0.000000
2023-10-30 08:15:00+00:00   -0.000202
2023-10-30 08:30:00+00:00   -0.000313
2023-10-30 08:45:00+00:00    0.000085
2023-10-30 09:00:00+00:00    0.001261
Name: close, dtype: float64

In [15]:
df_tickers=df_tickers.assign(ewm12=df_tickers['close'].ewm(halflife=12).mean())
df_tickers.head()

,close,ewm12
timestamp,,
2023-10-30 08:00:00+00:00,168.90,168.900000
2023-10-30 08:15:00+00:00,168.85,168.874278
2023-10-30 08:30:00+00:00,168.85,168.865714
2023-10-30 08:45:00+00:00,168.99,168.899527
2023-10-30 09:00:00+00:00,169.23,168.973469


In [16]:
df_tickers=df_tickers.assign(ewm26=df_tickers['close'].ewm(halflife=26).mean())
df_tickers.head()

,close,ewm12,ewm26
timestamp,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315


In [17]:
df_tickers=df_tickers.assign(macd=MACD_line)
df_tickers.head()

,close,ewm12,ewm26,macd
timestamp,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153


In [18]:
df_tickers=df_tickers.assign(signal=Signal_line)
df_tickers.head()

,close,ewm12,ewm26,macd,signal
timestamp,,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389,-0.000202
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511,-0.000313
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111,0.000085
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153,0.001261


In [19]:
df_tickers=df_tickers.assign(histogram=MACD_line-Signal_line)
df_tickers.head()

,close,ewm12,ewm26,macd,signal,histogram
timestamp,,,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000,0.000000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389,-0.000202,-0.000187
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511,-0.000313,-0.000198
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111,0.000085,0.001025
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153,0.001261,0.003893


In [20]:
Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Close_plot

:Curve   [timestamp]   (close)

In [21]:
MACD_plot = df_tickers['macd'].hvplot.line(    
    x='timestamp', 
    y='macd',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [22]:
Signal_plot = df_tickers['signal'].hvplot.line(    
    x='timestamp', 
    y='signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [23]:
plot = MACD_plot * Signal_plot 

plot

:Overlay
   .Curve.Macd   :Curve   [timestamp]   (macd)
   .Curve.Signal :Curve   [timestamp]   (signal)

In [24]:
Histogram_plot = df_tickers['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Histogram_plot

:Bars   [timestamp]   (histogram)

In [25]:
# itterating over MACD line and Signal Line values of the same date in the final Data Frame above
df_tickers['return']=df_tickers['close'].pct_change()
display(df_tickers)




,close,ewm12,ewm26,macd,signal,histogram,return
timestamp,,,,,,,
2023-10-30 08:00:00+00:00,168.9000,168.900000,168.900000,0.000000,0.000000,0.000000,NaN
2023-10-30 08:15:00+00:00,168.8500,168.874278,168.874667,-0.000389,-0.000202,-0.000187,-0.000296
2023-10-30 08:30:00+00:00,168.8500,168.865714,168.866224,-0.000511,-0.000313,-0.000198,0.000000
2023-10-30 08:45:00+00:00,168.9900,168.899527,168.898416,0.001111,0.000085,0.001025,0.000829
2023-10-30 09:00:00+00:00,169.2300,168.973469,168.968315,0.005153,0.001261,0.003893,0.001420
...,...,...,...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,169.891672,169.769723,0.121949,0.125345,-0.003396,-0.000177
2023-10-30 23:00:00+00:00,169.8200,169.887527,169.771369,0.116158,0.124658,-0.008500,-0.000294
2023-10-30 23:15:00+00:00,169.8500,169.885360,169.773928,0.111433,0.123669,-0.012236,0.000177


In [26]:
df_tickers["X"]=np.where(df_tickers["macd"]>df_tickers["signal"],1,0) # Creating signals
df_tickers["X"]=df_tickers["X"].diff() # Finding long or short trades
Xreturns=df_tickers[df_tickers["X"].isin([1,-1])] # Extracting just the trades where 1 is long, -1 is short
Xreturns.reset_index(inplace=True)
trades = Xreturns[["X","timestamp","close"]] # Dataframe with trade signals and dates
trades["sum"]=trades["close"].diff()
profits=abs(trades["sum"]).sum()

/Users/pigeoneyevideography/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
trades.head()

,X,timestamp,close,sum
0,1.0,2023-10-30 08:45:00+00:00,168.9900,NaN
1,-1.0,2023-10-30 12:00:00+00:00,168.9300,-0.0600
2,1.0,2023-10-30 13:30:00+00:00,170.8939,1.9639
3,-1.0,2023-10-30 22:45:00+00:00,169.8699,-1.0240


In [28]:
print(f"You made profits of ${profits}")

You made profits of $3.0478999999999985


In [29]:
X_plot = trades['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='Decission Making Points', 
    xlabel='timestamp',
    title='Long and Short Signals',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

X_plot

:Curve   [timestamp]   (close)